# extract face from id

In [9]:
from typing import Optional
from mtcnn import MTCNN
import cv2
import os
import numpy as np


def extract_and_save_face(image_path: str, *, save_path: str = "cropped_faces/", padding: float = 0.35) -> bool:
    """
    Detects a face in the given image, applies padding to the cropped face, saves it, and returns whether a face was found.

    Args:
        image_path (str): Path to the input image.
        save_path (str, keyword-only): Directory where cropped face images will be saved.
        padding (float, keyword-only): Padding factor (0.1 = 10% extra around the face).

    Returns:
        bool: True if a face was found and saved, False otherwise.
    """
    # Load image
    image: Optional[np.ndarray] = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return False

    # Initialize MTCNN detector
    detector = MTCNN()

    # Detect faces
    faces = detector.detect_faces(image)

    if not faces:
        print("No face detected.")
        return False

    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)

    height, width, _ = image.shape  # Image dimensions

    for i, face in enumerate(faces):  # Loop in case there are multiple faces
        x, y, w, h = face["box"]

        # Compute padding
        pad_x = int(w * padding)
        pad_y = int(h * padding)

        # Expand bounding box
        x1, y1 = max(x - pad_x, 0), max(y - pad_y, 0)
        x2, y2 = min(x + w + pad_x, width), min(y + h + pad_y, height)

        # Crop the padded face
        cropped_face = image[y1:y2, x1:x2]

        # Save cropped face
        face_filename = os.path.join(save_path, f"face_{os.path.basename(image_path)}")
        cv2.imwrite(face_filename, cropped_face)
        print(f"✅ Face saved at: {face_filename} with padding {padding*100:.1f}%")
        return True  # Return True after saving the first detected face

    return False  # If no face was saved

In [11]:
success = extract_and_save_face("adhar_front.jpg")
print(f"Face extraction successful: {success}")

✅ Face saved at: cropped_faces/face_adhar_front.jpg with padding 35.0%
Face extraction successful: True


# esrGAN

In [13]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os


# Load Pretrained ESRGAN Model from TensorFlow Hub
srgan_model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")


def enhance_image(image_path: str, *, save_path: str = "enhanced_faces/") -> bool:
    """
    Enhances the quality of a face image using a Super-Resolution GAN (SRGAN).

    Args:
        image_path (str): Path to the cropped face image.
        save_path (str, keyword-only): Directory where enhanced images will be saved.

    Returns:
        bool: True if enhancement is successful, False otherwise.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return False

    # Convert to RGB and normalize (0-1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0
    #image/=255.0
    # Resize to match ESRGAN input requirements
    #image_resized = cv2.resize(image, (50, 50))  # Model expects 50x50 input
    image_resized = np.expand_dims(image, axis=0).astype(np.float32)

    # Run through the ESRGAN model
    enhanced = srgan_model(image_resized)[0]  # Get first output

    # ✅ Convert EagerTensor to NumPy before processing
    enhanced = np.array(enhanced)  
    enhanced = (enhanced * 255.0).clip(0, 255).astype(np.uint8)  # Process as usual

    # Ensure save directory exists
    os.makedirs(save_path, exist_ok=True)

    # Save enhanced image
    enhanced_filename = os.path.join(save_path, f"enhanced_{os.path.basename(image_path)}")
    cv2.imwrite(enhanced_filename, cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR))
    print(f"✅ Enhanced face saved at: {enhanced_filename}")

    return True

2025-02-04 17:00:11,208 - WARNING - From C:\Users\sanje\anaconda3\envs\kyc\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-02-04 17:00:12,854 - INFO - Using C:\Users\sanje\AppData\Local\Temp\tfhub_modules to cache modules.


2025-02-04 17:00:13,014 - WARNING - From C:\Users\sanje\anaconda3\envs\kyc\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

2025-02-04 17:00:13,026 - INFO - Downloading TF-Hub Module 'https://tfhub.dev/captain-pool/esrgan-tf2/1'.
2025-02-04 17:00:19,144 - INFO - Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB
2025-02-04 17:00:19,148 - INFO - Downloaded TF-Hub Module 'https://tfhub.dev/captain-pool/esrgan-tf2/1'.


2025-02-04 17:00:19,253 - WARNING - From C:\Users\sanje\anaconda3\envs\kyc\Lib\site-packages\tensorflow_hub\module_v2.py:126: The name tf.saved_model.load_v2 is deprecated. Please use tf.compat.v2.saved_model.load instead.

2025-02-04 17:00:25,471 - INFO - Fingerprint not found. Saved model loading will continue.
2025-02-04 17:00:25,472 - INFO - path_and_singleprint metric could not be logged. Saved model loading will continue.


In [15]:
success = enhance_image("cropped_faces/face_adhar_front.jpg")
print(f"Super-resolution enhancement successful: {success}")

✅ Enhanced face saved at: enhanced_faces/enhanced_face_adhar_front.jpg
Super-resolution enhancement successful: True


# Face Matching- FaceNet

In [17]:
from keras_facenet import FaceNet
import numpy as np
import cv2
from scipy.spatial.distance import cosine

def match_faces_with_facenet(image1_path: str, image2_path: str):
    """
    Compares two pre-processed faces and returns whether they match and the similarity score.

    Args:
        image1_path (str): Path to the first image (already cropped and enhanced).
        image2_path (str): Path to the second image (already cropped and enhanced).

    Returns:
        tuple: A tuple containing:
            - bool: True if the faces match, False otherwise.
            - float: The cosine similarity score between the two faces.
    """
    # Load the images using OpenCV (ensure they are valid)
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    
    if image1 is None:
        raise ValueError(f"Failed to load image at {image1_path}")
    if image2 is None:
        raise ValueError(f"Failed to load image at {image2_path}")

    # Resize the images to the size that FaceNet expects (160x160)
    image1 = cv2.resize(image1, (160, 160))
    image2 = cv2.resize(image2, (160, 160))

    # Convert images to RGB (FaceNet expects RGB images, OpenCV loads in BGR by default)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    # Add an extra batch dimension: from (160, 160, 3) to (1, 160, 160, 3)
    image1 = np.expand_dims(image1, axis=0)
    image2 = np.expand_dims(image2, axis=0)

    # Initialize the FaceNet model
    facenet = FaceNet()

    # Extract face embeddings
    embedding1 = facenet.embeddings(image1)[0]
    embedding2 = facenet.embeddings(image2)[0]

    # Compute the cosine distance between the two embeddings
    distance = cosine(embedding1, embedding2)

    # Define a threshold for matching (typically, 0.6 to 0.7 is a common threshold)
    threshold = 0.6

    # Determine if faces match
    faces_match = distance < threshold

    return faces_match, distance

In [19]:
# Example usage:
image1_path = "enhanced_faces/enhanced_face_adhar_front.jpg"
image2_path = "enhanced_faces/enhanced_face_live.jpg"

faces_match, similarity_score = match_faces_with_facenet(image1_path, image2_path)

print(f"Do the faces match? {faces_match}")
print(f"Similarity Score (distance): {similarity_score}")

2025-02-04 17:01:05,660 - INFO - Loading weights.
2025-02-04 17:01:05,662 - INFO - Looking for C:\Users\sanje/.keras-facenet\20180402-114759\20180402-114759-weights.h5


2025-02-04 17:01:10,147 - WARNING - From C:\Users\sanje\anaconda3\envs\kyc\Lib\site-packages\keras\src\backend\tensorflow\core.py:216: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Do the faces match? True
Similarity Score (distance): 0.5981652736663818


# OCR using easyocr 

### PAN

In [4]:
import cv2
import numpy as np
import easyocr
import re
import logging

# Configure logging (PEP 282)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


def preprocess_image(image_path: str) -> np.ndarray:
    """
    Preprocess the image by converting it to grayscale and applying adaptive thresholding.
    
    Args:
        image_path (str): Path to the image file.

    Returns:
        np.ndarray: Preprocessed thresholded image.
    """
    img = cv2.imread(image_path)

    if img is None:
        logging.error(f"Failed to load image: {image_path}")
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to enhance text visibility
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2
    )

    logging.info("Image preprocessing completed.")
    return thresh


def extract_pan_details(*, image_path: str) -> dict:
    """
    Extract important details from a PAN card image using OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: Extracted information including ID Number, Date of Birth, Name, and Father's Name.
    """
    reader = easyocr.Reader(["en"])  # Load OCR reader for English

    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Extract text using EasyOCR
    results = reader.readtext(processed_img, detail=0)  # Extract text only

    extracted_info = {
        "ID Number": None,
        "Date of Birth": None,
        "Name": None,
        "Father's Name": None,
    }

    name_found = False  # Track if Name is found
    father_name_found = False  # Track if Father's Name is found

    for i, text in enumerate(results):
        text = text.strip()

        # Extract PAN Number (Format: ABCDE1234F)
        if re.match(r"^[A-Z]{5}[0-9]{4}[A-Z]$", text):
            extracted_info["ID Number"] = text

        # Extract Date of Birth (Format: DD/MM/YYYY)
        if re.match(r"\d{2}/\d{2}/\d{4}", text):
            extracted_info["Date of Birth"] = text

        # Extract Name (Assuming it appears after "Name" keyword)
        if "name" in text.lower() and i + 1 < len(results) and not name_found:
            extracted_info["Name"] = results[i + 1].strip()
            name_found = True

        # Extract Father's Name (Appears after "Father's Name" or "पिता का नाम")
        if "father" in text.lower() and i + 1 < len(results) and not father_name_found:
            extracted_info["Father's Name"] = results[i + 1].strip()
            father_name_found = True

    logging.info(f"Extracted details: {extracted_info}")
    return extracted_info

### adhar card front

In [24]:
def ext(image_path):
    """
    Extract important details from an Aadhaar card image using OCR.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: Extracted information including Aadhaar Number, Name, Date of Birth, and Gender.
    """
    reader = easyocr.Reader(["en"])  # Load OCR reader for English & Hindi

    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Extract text using EasyOCR
    results = reader.readtext(processed_img, detail=0)  # Extract text only

    extracted_info = {
        "Aadhaar Number": None,
        "Date of Birth": None,
        "Gender": None,
    }

    name_candidates = []  # Track if Name is found
    gender_found = False  # Track if Gender is found
    
    for i, text in enumerate(results):
        text = text.strip()

        # Extract Aadhaar Number (Format: 12-digit number)
        if re.match(r"^\d{4}\s?\d{4}\s?\d{4}$", text):
            extracted_info["Aadhaar Number"] = text.replace(" ", "")  # Remove spaces

        # Extract Date of Birth (Format: DD/MM/YYYY)
        match=re.search(r"\b(\d{2}/\d{2}/\d{4})\b", text)
        if match:
            extracted_info["Date of Birth"] = match.group(1)



        # Extract Gender (Look for "Male", "Female" or "पुरुष", "महिला")
        if not gender_found:
            if "male" in text.lower():
                extracted_info["Gender"] = "Male"
                gender_found = True
            elif "female" in text.lower():
                extracted_info["Gender"] = "Female"
                gender_found = True

    return extracted_info

